<a href="https://colab.research.google.com/github/pri188/Capstone-3---Classify-Image-if-it-is-GenAI-or-Real/blob/main/Is_GenAI_Image_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [9]:
%pip install torch==2.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [10]:
%pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0
    Uninstalling torch-2.5.0:
      Successfully uninstalled torch-2.5.0


In [11]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8bbeza1x
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8bbeza1x
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=77c736b37114a7c5d295099cd7530a35c26a2d32c9ad87acc574d2c91fd5cc14
  Stored in directory: /tmp/pip-ephem-wheel-cache-yn9_ce7s/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [12]:
%pip install ftfy regex tqdm

# Read images and generate CLIP embeddings

In [34]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_image_embedddings(image_path):
  image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
  image_features = model.encode_image(image).detach().numpy().tolist()
  return image_features

In [36]:
from os import listdir
from os.path import isfile, join
import numpy as np

categories = ['real', 'genai']
embeddings = []
for category in categories:
  category_path = f'genai_data/{category}/'
  image_names = [f for f in listdir(category_path) if isfile(join(category_path, f))]
  print(image_names)
  for image_name in image_names:
    image_features = get_image_embedddings(f'{category_path}{image_name}')
    label = 0 if category == 'real' else 1
    image_features.append(image_name)
    image_features.append(label)
    embeddings.append(image_features)

['Saltwater Crocodile.jpeg', 'Zebra.jpeg', 'house.jpg', 'Mango_4.jpg', 'Lion.jpeg', 'Elephant.jpg', 'Baby.jpg', 'Eiffel Tower.jpeg', 'Real Apple.jpg', 'table_grapes_grapes_fruit.jpg', 'Parrot.jpg']
['Parrot.jpeg', 'Lion.jpg', 'Apple.jpg', 'baby.jpeg', 'Zebra.jpg', 'Mango_4.jpg', 'Lion.jpeg', 'table grapes.jpg', 'Elephant.jpg', 'Baby.jpg', 'Eiffel Tower.jpeg', 'Real Apple.jpg']


In [47]:
import pandas as pd

df = pd.DataFrame(embeddings, columns=['embeddings', 'image_name', 'label'])
df.head()

,embeddings,image_name,label
0,"[0.3451813757419586, -0.5790499448776245, -0.2...",Saltwater Crocodile.jpeg,0
1,"[0.30487000942230225, -0.22138716280460358, -0...",Zebra.jpeg,0
2,"[-0.04642140120267868, 0.2565220892429352, 0.1...",house.jpg,0
3,"[0.2503420412540436, 0.28583136200904846, -0.0...",Mango_4.jpg,0
4,"[0.03871428593993187, -0.39236465096473694, -0...",Lion.jpeg,0


# Convert labels and features to train test set

In [105]:
from sklearn.model_selection import train_test_split

total_repeatation = 10000
df_concat = pd.concat([df] * total_repeatation, ignore_index=True)

if 'image_name' in df_concat.columns:
  df_concat = df_concat.drop('image_name', axis=1)
X = df_concat.drop('label', axis=1)
y = df_concat['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.array(X_train['embeddings'].to_list(), dtype=np.float32)
X_train = torch.from_numpy(X_train)

X_test = np.array(X_test['embeddings'].to_list(), dtype=np.float32)
X_test = torch.from_numpy(X_test)

y_train = torch.tensor(y_train.to_list(), dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test.to_list(), dtype=torch.float32).reshape(-1, 1)

# Train a model using pytorch Neural Network

In [106]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np

# Generate random data for testing (do not use)
'''X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)'''

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassifier, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 256
model = BinaryClassifier(input_size, hidden_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 100
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.3224
Epoch [20/100], Loss: 0.3053
Epoch [30/100], Loss: 0.3030
Epoch [40/100], Loss: 0.3021
Epoch [50/100], Loss: 0.3018
Epoch [60/100], Loss: 0.3017
Epoch [70/100], Loss: 0.3017
Epoch [80/100], Loss: 0.3017
Epoch [90/100], Loss: 0.3017
Epoch [100/100], Loss: 0.3017


# evaluate the model

In [107]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = (y_pred.round())
    accuracy = (y_pred_class == y_test).float().mean()
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.7824
